In [1]:
import pandas as pd
import numpy as np

In [3]:
np.random.seed(456)
pandas_dataframe = pd.DataFrame(dict(n=np.arange(20), group = np.random.choice(list('abc'), 20))
                               )
pandas_dataframe

,n,group
0,0,b
1,1,b
2,2,c
3,3,a
4,4,c
5,5,c
6,6,a
7,7,b
8,8,a
9,9,b


In [5]:
import pyspark
spark = pyspark.sql.SparkSession.buider.getOrCreate()

AttributeError: type object 'SparkSession' has no attribute 'buider'

In [4]:
df = spark.createDataFrame(pandas_dataframe)
df

NameError: name 'spark' is not defined